In [1]:
import sys
sys.path.append("..")

In [2]:
import re
from collections import defaultdict
from resources.utils import get_puzzle_input

### Part 1

The instructions specify a series of steps and requirements about which steps must be finished before others can begin (your puzzle input). Each step is designated by a single letter. For example, suppose you have the following instructions:

```
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
```

Visually, these requirements look like this:

```
  -->A--->B--
 /    \      \
C      -->D----->E
 \           /
  ---->F-----
```

Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

Only C is available, and so it is done first.
Next, both A and F are available. A is first alphabetically, so it is done next.
Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.
After that, only D and F are available. E is not available because only some of its prerequisites are complete. Therefore, D is completed next.
F is the only choice, so it is done next.
Finally, E is completed.
So, in this example, the correct order is CABDFE.

In what order should the steps in your instructions be completed?

In [3]:
# Step F must be finished before step E can begin.
step_re = re.compile(r'Step (.) must be finished before step (.) can begin.')

In [4]:
assert step_re.match('Step F must be finished before step E can begin.').groups() == ('F', 'E')

In [5]:
test_input = [
    'Step C must be finished before step A can begin.',
    'Step C must be finished before step F can begin.',
    'Step A must be finished before step B can begin.',
    'Step A must be finished before step D can begin.',
    'Step B must be finished before step E can begin.',
    'Step D must be finished before step E can begin.',
    'Step F must be finished before step E can begin.',
]

In [6]:
def parse_input(steps):
    ancestors = defaultdict(list)
    decendents = defaultdict(list)
    
    for step in steps:
        ancestor, decendent = step_re.match(step).groups()
        
        if ancestor not in ancestors:
            ancestors[ancestor] = []
            
        decendents[ancestor].append(decendent)
        ancestors[decendent].append(ancestor)
    
    return ancestors, decendents   

In [7]:
parse_input(test_input)

(defaultdict(list,
             {'C': [],
              'A': ['C'],
              'F': ['C'],
              'B': ['A'],
              'D': ['A'],
              'E': ['B', 'D', 'F']}),
 defaultdict(list,
             {'C': ['A', 'F'],
              'A': ['B', 'D'],
              'B': ['E'],
              'D': ['E'],
              'F': ['E']}))

In [8]:
def solve(input):
    ancestors, decendents = parse_input(input)
    process_items = []
    
    while len(ancestors):
        ready_items = [item for (item, ancestors) in ancestors.items() if not ancestors]
        process_item = sorted(ready_items)[0]
        process_items.append(process_item)
        del ancestors[process_item]

        for decendent in decendents[process_item]:
            ancestors[decendent].remove(process_item)
        
    return ''.join(process_items)  

In [9]:
assert solve(test_input) == 'CABDFE'

In [10]:
puzzle_input = get_puzzle_input('/tmp/day7.txt')

In [11]:
solve(puzzle_input)

'EFHLMTKQBWAPGIVXSZJRDUYONC'

### Part 2


Your puzzle answer was EFHLMTKQBWAPGIVXSZJRDUYONC.

The first half of this puzzle is complete! It provides one gold star: *

--- Part Two ---
As you're about to begin construction, four of the Elves offer to help. "The sun will set soon; it'll go faster if we work together." Now, you need to account for multiple people working on steps simultaneously. If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:

```
Second   Worker 1   Worker 2   Done
   0        C          .        
   1        C          .        
   2        C          .        
   3        A          F       C
   4        B          F       CA
   5        B          F       CA
   6        D          F       CAB
   7        D          F       CAB
   8        D          F       CAB
   9        D          .       CABF
  10        E          .       CABFD
  11        E          .       CABFD
  12        E          .       CABFD
  13        E          .       CABFD
  14        E          .       CABFD
  15        .          .       CABFDE
  
  ```
  
Each row represents one second of time. The Second column identifies how many seconds have passed as of the beginning of that second. Each worker column shows the step that worker is currently doing (or . if they are idle). The Done column shows completed steps.

Note that the order of the steps has changed; this is because steps now take time to finish and multiple workers can begin multiple steps simultaneously.

In this example, it would take 15 seconds for two workers to complete these steps.

With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?

In [12]:
class AssemblyInstructions:
    def __init__(self, steps):
        self.ancestors, self.decendents = parse_input(steps)
        self.steps_in_progress = set()
    
    @property
    def is_complete(self):
        return not self.ancestors and not self.steps_in_progress

    def get_step(self):
        ready_items = [
            item for (item, ancestors)
            in self.ancestors.items()
            if not ancestors and item not in self.steps_in_progress
        ]
        
        if not ready_items:
            return None

        next_step = sorted(ready_items)[0]

        self.steps_in_progress.add(next_step)
        return next_step
        
    def complete_step(self, step):
        self.steps_in_progress.remove(step)
        del self.ancestors[step]
        
        for decendent in self.decendents[step]:
            self.ancestors[decendent].remove(step)
        

In [13]:
test_assembly = AssemblyInstructions(test_input)

while not test_assembly.is_complete:
    step = test_assembly.get_step()
    print(step)
    test_assembly.complete_step(step)   

C
A
B
D
F
E


In [14]:
def processing_time(step, offset=0):
    return ord(step) - 64 + offset   

In [15]:
assert processing_time('A') == 1
assert processing_time('Z') == 26
assert processing_time('A', 60) == 61

In [16]:
class AssemblyWorker:
    def __init__(self, ident, instructions, time_offset, log_activity=False):
        self.ident = ident
        self.instructions = instructions
        self.time_offset = time_offset
        self.time_remaining = 0
        self.log_activity = log_activity
        self.in_progress = None
        
    def _log(self, action):
        if self.log_activity:
            print('WORKER {}: {}'.format(self.ident, action))
            
    @property
    def state(self):
        return self.in_progress or '.'
    
    def pick_up(self):
        if self.in_progress:
            # Already processing
            return

        step = self.instructions.get_step()
        
        if step:
            self._log('Got task {}'.format(step))
            self.time_remaining = processing_time(step, self.time_offset)
            self.in_progress = step
        else:
            self._log('No ready tasks')
    
    def process(self):
        if not self.in_progress:
            return

        self.time_remaining -= 1
        if self.time_remaining:
            return

        self._log('Completed task {}'.format(self.in_progress))
        self.instructions.complete_step(self.in_progress)
        self.in_progress = None

    def act(self):
        '''Original WRONG solution (but worked when submitted with output-1!)
        
        Means that a worker can only pick up a dependent task if the dependency
        was completed by a prior worker when iterating over the set.
        
        Some workers are more equal than others...
        '''
        self.process()
        self.pick_up()



In [17]:
def assembly_line(
    instruction_set,
    num_workers,
    offset,
    log_worker=False,
    log_time=False,
    correct_solution=True,
    max_time=0
):
    instructions = AssemblyInstructions(instruction_set)
    
    workers = [AssemblyWorker(x, instructions, offset, log_worker) for x in range(num_workers)]
    
    time = 0    
    while not instructions.is_complete:
        if log_worker:
            print('SECOND ', time)
        
        if correct_solution:  
            for worker in workers:
                worker.pick_up()
            states = [worker.state for worker in workers]
            for worker in workers:
                worker.process()
        else:
            for worker in workers:
                worker.act()
            states = [worker.state for worker in workers]
        
        if log_time:
            print('{}\t{}'.format(time, ' '.join(states)))
        
        time += 1
        if max_time and time > max_time:
            print('Over allowed time')

    return time

In [18]:
assembly_line(test_input, 2, 0, correct_solution=True, log_time=True)

0	C .
1	C .
2	C .
3	A F
4	B F
5	B F
6	D F
7	D F
8	D F
9	D .
10	E .
11	E .
12	E .
13	E .
14	E .


15

In [19]:
assembly_line(puzzle_input, 5, 60, correct_solution=True)

1056